In [10]:
sentences = [
    'The RTX3080 is a marvel to behold.',
    'This model"s price has decreased compared to previous models.',
    'The temperature of this model is a bit hot.',
    'The cooling is honestly terrible.',
    'The next installment in the RTX-series is one of a kind.',
    'The RTX 3080 is a beast.'
]

In [14]:
import nltk

In [17]:
global dtn
global adjectives
adjectives = []
global all_sentnc
all_sentnc = []

def processing(id, content):
    global dtn
    global adjectives
    global all_sentnc

    sentnc_list = []
    sentnc_list_NN = []
    sentnc_list_JJ = []

    sentence_info = []
    sentence_info.append(id)
    sentence_info.append(content)

    all_sentnc.append(sentence_info)
    sentence_info = []

    tokenized = nltk.word_tokenize(content)
    tagged_list = nltk.pos_tag(tokenized)

    for entry in tagged_list:
        if "JJ" in entry:
            sentnc_list_JJ.append(entry[0])
        elif "NN" in entry or "NNS" in entry or "NNP" in entry or "NNPS" in entry:
            sentnc_list_NN.append(entry[0])

    sentnc_list.append(sentnc_list_NN)
    sentnc_list.append(sentnc_list_JJ)

    adjectives.append(sentnc_list)

In [21]:
def getSentences():
    return all_sentnc